In [1]:
import numpy as np
%load_ext line_profiler

In [2]:
%%writefile simulation.py
import numpy as np

def step(*shape):
    # Create a random n-vector with +1 or -1 values.
    return 2 * (np.random.random_sample(shape)<.5) - 1

def simulate(iterations, n=10000):
    s = step(iterations, n)
    x = np.cumsum(s, axis=0)
    bins = np.arange(-30, 30, 1)
    y = np.vstack([np.histogram(x[i,:], bins)[0]
                   for i in range(iterations)])
    return y

Overwriting simulation.py


In [3]:
from simulation import simulate

In [4]:
%lprun -T lprof0 -f simulate simulate(50)


*** Profile printout saved to text file 'lprof0'. 


In [5]:
print(open('lprof0', 'r').read())


Timer unit: 1e-06 s

Total time: 0.063828 s
File: /mnt/c/Users/Test/ml/simulation.py
Function: simulate at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def simulate(iterations, n=10000):
     8         1      31489.0  31489.0     49.3      s = step(iterations, n)
     9         1       8410.0   8410.0     13.2      x = np.cumsum(s, axis=0)
    10         1         39.0     39.0      0.1      bins = np.arange(-30, 30, 1)
    11         1          3.0      3.0      0.0      y = np.vstack([np.histogram(x[i,:], bins)[0]
    12         1      23885.0  23885.0     37.4                     for i in range(iterations)])
    13         1          2.0      2.0      0.0      return y


In [6]:
#!pip  install py-heat-magic 

In [1]:
import os
from tempfile import mkstemp

from IPython.core import magic_arguments
from IPython.core.magic import Magics, cell_magic, magics_class
from pyheat import PyHeat
%load_ext heat

The heat module is not an IPython extension.


In [ ]:
%%heat

In [ ]:
@magics_class
class PyHeatMagic(Magics):
    """Class with IPython magic commands to effectively use py-heat profiling
    within IPython."""

    @magic_arguments.magic_arguments()
    @magic_arguments.argument(
        "-o", "--out", default=None, help="Save the heatmap to given file"
    )
    @cell_magic
    def heat(self, line, cell):
        """Method to profile the python code in the ipython cell and display it
        as a heatmap using py-heat package.

        :param line: Line value for the ipython line this magic is called from.
        :param cell: Cell value for the ipython cell this magic is called from.
        """
        args = magic_arguments.parse_argstring(self.heat, line)
        filename = args.out
        if filename is not None:
            filename = os.path.expanduser(args.out)

        _, tmp_file = mkstemp()
        with open(tmp_file, "wb") as f:
            f.write(cell.encode())

        pyheat = PyHeat(tmp_file)
        pyheat.create_heatmap()
        pyheat.show_heatmap(output_file=filename)
        pyheat.close_heatmap()

        os.remove(tmp_file)


def load_ipython_extension(ipython):
    ipython.register_magics(PyHeatMagic)
